# Recommender system
## Collaborative filtering


In [1]:
# importing the neessary libs
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
def open_create_list(path, sep, column):
    """Function that opens a file and extract a specific column from it creating a list."""
    var = open(path, 'r', encoding='iso-8859-1')  
    new_list = []
    for l in var:
        l = l.split(sep)
        new_list.append(l[column])
    return new_list

### Creating a DataFrame with the important columns from the files

In [3]:
# Columns from u.data: | user id | item id | rating | timestamp |
userId_data = open_create_list('data/u.data', '\t', 0)
movieId_data = open_create_list('data/u.data', '\t', 1)
rating_data = open_create_list('data/u.data', '\t', 2)

# creating an array with the lists data
array_data = np.array([userId_data, movieId_data, rating_data])

In [7]:
# creating a dataframe with the data
data_df = pd.DataFrame(data = array_data)
data_df

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
0,196,186,22,244,166,298,115,253,305,6,...,806,676,721,913,378,880,716,276,13,12
1,242,302,377,51,346,474,265,465,451,86,...,421,538,262,209,78,476,204,1090,225,203
2,3,3,1,2,1,4,2,5,3,3,...,4,4,3,2,3,3,5,1,2,3


In [8]:
# transposing the DF and setting the proper column name
data_df = data_df.T
data_df.columns = 'userID movieID rating'.split()
data_df.head()

,userID,movieID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [10]:
# loading the unique movie information
# u.item first 3 columns: | movie id | movie title | release date |
movieId_uniq = open_create_list('data/u.item', '|', 0)
movieName_uniq = open_create_list('data/u.item', '|', 1)
movieDate_uniq = open_create_list('data/u.item', '|', 2)

array_item = np.array([movieId_uniq, movieName_uniq, movieDate_uniq])

In [12]:
# creating a dataframe with the unique movie information 
item_uniq_df = pd.DataFrame(data= array_item)
item_uniq_df = item_uniq_df.T
item_uniq_df.columns = 'movieID Name Date'.split()
item_uniq_df.head()

,movieID,Name,Date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
